In [1]:
import numpy as np
import pandas as pd
import datetime

### 기존 데이터프레임

In [2]:
batter = pd.read_csv('private_batter.csv')
batter = batter.sort_values(by=['P_ID','GDAY_DS'])
batter['GDAY_DS'] = pd.to_datetime(batter['GDAY_DS'],format='%Y-%m-%d')

In [3]:
pitcher = pd.read_csv('private_pitcher.csv')
pitcher = pitcher.sort_values(by=['P_ID','GDAY_DS'])
pitcher['GDAY_DS'] = pd.to_datetime(pitcher['GDAY_DS'],format='%Y-%m-%d')

### 타자 데이터프레임 변형

- train 데이터

In [4]:
df=pd.DataFrame()
for i in batter.P_ID.unique():
    temp = batter[batter['P_ID']==i].drop('AVG',axis=1)
    temp = temp.reset_index(drop='index')
    future_y = temp[['AB','HIT']]
    x = temp.drop(['AB','HIT'],axis=1)
    x = x.shift(periods=1,axis=0)
    new = pd.concat([x,future_y],axis=1)
    new = new.drop(0)
    df = pd.concat([df,new])

In [5]:
df = df.reset_index(drop='index')

In [6]:
df.to_csv('new_dataframe/batter_train.csv',index=False)

- test x

In [7]:
final_x = pd.DataFrame()
for i in batter.P_ID.unique():
    temp = batter[batter['P_ID']==i].drop(['AVG','AB','HIT'],axis=1).tail(1)
    final_x = pd.concat([final_x,temp])

In [8]:
final_x = final_x[final_x['GDAY_DS'].dt.year>=2020]
df = pd.concat([pd.get_dummies(df['TB_SC']),df],axis=1)
final_x = pd.concat([pd.get_dummies(final_x['TB_SC']),final_x],axis=1)

In [9]:
final_x.to_csv('new_dataframe/batter_test.csv',index=False)

### 투수 데이터프레임 변형

- train 데이터

In [10]:
df=pd.DataFrame()
for i in pitcher.P_ID.unique():
    temp = pitcher[pitcher['P_ID']==i].drop('ERA',axis=1)
    temp = temp.reset_index(drop='index')
    future_y = temp[['INN2','ER']]
    x = temp.drop(['INN2','ER'],axis=1)
    x = x.shift(periods=1,axis=0)
    new = pd.concat([x,future_y],axis=1)
    new = new.drop(0)
    df = pd.concat([df,new])

In [11]:
df = df.reset_index(drop='index')

In [12]:
df.to_csv('new_dataframe/pitcher_train.csv',index=False)

- test x 

In [13]:
final_x = pd.DataFrame()
for i in pitcher.P_ID.unique():
    temp = pitcher[pitcher['P_ID']==i].drop(['INN2','ER','ERA'],axis=1).tail(1)
    final_x = pd.concat([final_x,temp])

In [14]:
final_x = final_x[final_x['GDAY_DS'].dt.year>=2020]
df = pd.concat([pd.get_dummies(df['TB_SC']),df],axis=1)
final_x = pd.concat([pd.get_dummies(final_x['TB_SC']),final_x],axis=1)

In [15]:
final_x.to_csv('new_dataframe/pitcher_test.csv',index=False)